# レスラー方程式（外力のある場合）

外力のない場合のレスラー方程式の予測はそれなりにうまく行った気がする．
次は外力のある場合のレスラー方程式の予測を行ってみる．

注．hyperoptが同じhyperparametersを返してしまう問題だが，train_lenを10000にすることによって回避できた．
単純に学習期間が不足していたのかもしれない．

In [2]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time


CPU times: user 4 µs, sys: 9 µs, total: 13 µs
Wall time: 24.3 µs


In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd

# 外力P(t)の振幅Aをパラメータとして追加します。
A = 0.1  # 外力の振幅を設定します。この値を変更することで、外力の強さを変えられます。

# レスラー方程式の定義を変更して、外力P(t)を加えます。
def rossler_system_with_force(t, state, a, b, c):
    x, y, z = state
    dxdt = -y - z + A * np.sin(t)  # X項に外力P(t)を加える
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]

# パラメータ
a = 0.2
b = 0.2
c = 5.7

# 初期条件
initial_state = [1.0, 1.0, 1.0]

# 時間の設定
t_span = [0, 2510]  # 開始時刻と終了時刻
t_eval = np.linspace(t_span[0], t_span[1], 25100)  # 評価する時間点


# 微分方程式の数値解を求める
solution_with_force = solve_ivp(rossler_system_with_force, t_span, initial_state, args=(a, b, c), t_eval=t_eval, max_step=0.01)

# 外力P(t)の計算
P = A * np.sin(solution_with_force.t)

# CSVファイルにデータを保存するためのDataFrameを作成
data_frame_with_force = pd.DataFrame({
    'time': solution_with_force.t,
    'x': solution_with_force.y[0],
    'y': solution_with_force.y[1],
    'z': solution_with_force.y[2],
    'P': P  # 外力P(t)の列を追加
})

# CSVファイルにデータを保存
filename_with_force = 'rossler_data_with_force1.3.1.csv'
data_frame_with_force.to_csv(filename_with_force, index=False)

# CSVファイルを読み込む
data_loaded_with_force = pd.read_csv(filename_with_force)

# CSVから値を抽出してNumpy配列に格納
X = data_loaded_with_force[['x', 'y', 'P']].values

print(X)


[[1.         1.         0.        ]
 [0.8143561  1.11164722 0.00998374]
 [0.64878658 1.20790655 0.01986771]
 ...
 [3.25371407 5.93023934 0.03264432]
 [2.60596253 6.34638839 0.02304443]
 [1.92641531 6.70386756 0.01321426]]
